In [305]:
import pandas as pd
import numpy as np
import random
import pyarrow

## Generate Data

In [ ]:
# Guess species using name or breed?

In [128]:
print(data["message"]["bulldog"])

['boston', 'english', 'french']


In [136]:
import urllib.request
import json

## Load possible data values

In [279]:
# Get sample data for registered pets in Seattle
df = pd.read_json("https://data.seattle.gov/resource/87w5-fd6d.json");
df.head()

,animal_s_name,license_issue_date,license_number,primary_breed,secondary_breed,species,zip_code
0,Tinkerdelle,2003-04-19T00:00:00.000,200097,Domestic Shorthair,NaN,Cat,98116
1,Pepper,2006-02-07T00:00:00.000,75432,Manx,Mix,Cat,98103
2,Buddy,2008-03-20T00:00:00.000,135598,"Retriever, Labrador",Mix,Dog,98133
3,Finigan,2008-12-01T00:00:00.000,957147,"Spaniel, Irish Water",NaN,Dog,98107
4,Jenkey,2011-03-29T00:00:00.000,141222,Australian Cattle Dog,Mix,Dog,98106


In [280]:
# Build hierarchical source dataset - name and breed depend on species
# This isn't very necessary -- we could have just taken x records from the sample set
sourceData = {};

for species in df["species"].unique():
    speciesData = df[df["species"] == species];

    breeds = [x.title() for x in speciesData["primary_breed"].unique() if x.lower().count("mixed") == 0];
    names = [x.title() for x in speciesData["animal_s_name"].unique() if type(x) == str];

    sourceData[species] = {
        "breeds" : breeds,
        "names" : names
    }

## Build sample set

In [283]:
records = pd.DataFrame(columns=["Name", "Species", "Breed"]);

numRecords = 30;
for i in range(0, numRecords):
    species = random.choice(list(sourceData.keys()));
    records.loc[i] = {
        "Name" : random.choice(sourceData[species]["names"]),
        "Species" : species,
        "Breed" : random.choice(sourceData[species]["breeds"])
    };

records.head()

,Name,Species,Breed
0,Macchiatto,Cat,Devon Rex
1,Penny,Dog,Miniature Pinscher
2,Jameson,Dog,Japanese Chin
3,Dani,Dog,"Retriever, Golden"
4,Macchiatto,Cat,Balinese


In [308]:
records.set_index("Name").to_parquet("pets.parquet")

## Build a graph
* Create a node for each record
* Create a node for each field-value
* Link records to their field-values

In [284]:
from pyspark import *
from pyspark.sql import *

In [288]:
sdf_records = spark.createDataFrame(records)

In [302]:
a = list(records["Name"].unique())
b = list(records["Breed"].unique())

In [287]:
sdf_values = spark.createDataFrame()

DataFrame[Name: string, Species: string, Breed: string]